## 1. Import Required Libraries

In [1]:
import os
import numpy as np
import pandas as pd
import cv2
from PIL import Image
import matplotlib.pyplot as plt
import seaborn as sns
import albumentations as A
from albumentations.pytorch import ToTensorV2
import torch
from torch.utils.data import Dataset, DataLoader
import warnings
warnings.filterwarnings('ignore')

print("All libraries imported successfully!")
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")

d:\Semantic_Segmentation\offroad-env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


All libraries imported successfully!
PyTorch version: 2.9.1+cpu
CUDA available: False


## 2. Define Paths and Configuration

In [2]:
# Dataset paths
TRAIN_IMG_PATH = r'D:\Semantic_Segmentation\Offroad_Segmentation_Training_Dataset\Offroad_Segmentation_Training_Dataset\train\Color_Images'
TRAIN_MASK_PATH = r'D:\Semantic_Segmentation\Offroad_Segmentation_Training_Dataset\Offroad_Segmentation_Training_Dataset\train\Segmentation'
VAL_IMG_PATH = r'D:\Semantic_Segmentation\Offroad_Segmentation_Training_Dataset\Offroad_Segmentation_Training_Dataset\val\Color_Images'
VAL_MASK_PATH = r'D:\Semantic_Segmentation\Offroad_Segmentation_Training_Dataset\Offroad_Segmentation_Training_Dataset\val\Segmentation'

# Class mapping
CLASS_MAPPING = {
    100: 0,    # Trees
    200: 1,    # Lush Bushes
    300: 2,    # Dry Grass
    500: 3,    # Dry Bushes
    550: 4,    # Ground Clutter
    600: 5,    # Flowers
    700: 6,    # Logs
    800: 7,    # Rocks
    7100: 8,   # Landscape
    10000: 9   # Sky
}

CLASS_NAMES = {
    0: 'Trees',
    1: 'Lush Bushes',
    2: 'Dry Grass',
    3: 'Dry Bushes',
    4: 'Ground Clutter',
    5: 'Flowers',
    6: 'Logs',
    7: 'Rocks',
    8: 'Landscape',
    9: 'Sky'
}

# Configuration
CONFIG = {
    'img_size': (512, 512),
    'num_classes': 10,
    'batch_size': 16,
    'num_workers': 0,
    'seed': 42
}

print(f"Configuration: {CONFIG}")

Configuration: {'img_size': (512, 512), 'num_classes': 10, 'batch_size': 16, 'num_workers': 0, 'seed': 42}


## 3. Calculate Normalization Statistics

In [3]:
def calculate_normalization_params(img_dir, num_samples=100):
    """Calculate mean and std for normalization"""
    images = sorted([f for f in os.listdir(img_dir) if f.endswith('.png')])[:num_samples]
    
    all_means = []
    all_stds = []
    
    for img_file in images:
        img = cv2.imread(os.path.join(img_dir, img_file))
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = img / 255.0
        
        all_means.append(np.mean(img, axis=(0, 1)))
        all_stds.append(np.std(img, axis=(0, 1)))
    
    mean = np.mean(all_means, axis=0)
    std = np.mean(all_stds, axis=0)
    
    return mean, std

# Calculate normalization parameters
print("Calculating normalization parameters...")
mean, std = calculate_normalization_params(TRAIN_IMG_PATH, num_samples=100)

print(f"\nNormalization Statistics:")
print(f"Mean: {mean}")
print(f"Std: {std}")

NORM_MEAN = mean
NORM_STD = std

Calculating normalization parameters...

Normalization Statistics:
Mean: [0.51130156 0.42428829 0.3174731 ]
Std: [0.30373993 0.27931814 0.27089236]

Normalization Statistics:
Mean: [0.51130156 0.42428829 0.3174731 ]
Std: [0.30373993 0.27931814 0.27089236]


## 4. Augmentation Pipelines
train_transform = A.Compose([
    A.Resize(CONFIG['img_size'][0], CONFIG['img_size'][1]),
    A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.3),
    A.Rotate(limit=45, p=0.5),
    A.GaussNoise(p=0.2),
    A.GaussianBlur(p=0.2),
    A.RandomBrightnessContrast(p=0.3),
    A.Normalize(mean=NORM_MEAN, std=NORM_STD),
    ToTensorV2()
])

val_transform = A.Compose([
    A.Resize(CONFIG['img_size'][0], CONFIG['img_size'][1]),
    A.Normalize(mean=NORM_MEAN, std=NORM_STD),
    ToTensorV2()
])

print("Data augmentation pipelines created:")
print(f"- Training augmentations: {len(train_transform)}")
print(f"- Validation augmentations: {len(val_transform)}")

## 5. Custom Dataset Class

In [4]:
train_transform = A.Compose([
    A.Resize(CONFIG['img_size'][0], CONFIG['img_size'][1]),
    A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.3),
    A.Rotate(limit=45, p=0.5),
    A.GaussNoise(p=0.2),
    A.GaussianBlur(p=0.2),
    A.RandomBrightnessContrast(p=0.3),
    A.Normalize(mean=NORM_MEAN, std=NORM_STD),
    ToTensorV2()
])

val_transform = A.Compose([
    A.Resize(CONFIG['img_size'][0], CONFIG['img_size'][1]),
    A.Normalize(mean=NORM_MEAN, std=NORM_STD),
    ToTensorV2()
])

print("Data augmentation pipelines created:")
print(f"- Training augmentations: {len(train_transform)}")
print(f"- Validation augmentations: {len(val_transform)}")

Data augmentation pipelines created:
- Training augmentations: 9
- Validation augmentations: 3


In [5]:
class SegmentationDataset(Dataset):
    """Custom dataset for semantic segmentation"""
    
    def __init__(self, img_dir, mask_dir, transform=None):
        self.img_dir = img_dir
        self.mask_dir = mask_dir
        self.transform = transform
        self.images = sorted([f for f in os.listdir(img_dir) if f.endswith('.png')])
    
    def __len__(self):
        return len(self.images)
    
    def __getitem__(self, idx):
        img_name = self.images[idx]
        
        # Load image
        img_path = os.path.join(self.img_dir, img_name)
        image = cv2.imread(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        # Load mask
        mask_path = os.path.join(self.mask_dir, img_name)
        mask = np.array(Image.open(mask_path))
        
        # Convert mask class IDs to indices
        mask_indexed = np.zeros_like(mask, dtype=np.int64)
        for class_id, idx_val in CLASS_MAPPING.items():
            mask_indexed[mask == class_id] = idx_val
        
        # Apply augmentation
        if self.transform:
            augmented = self.transform(image=image, mask=mask_indexed)
            image = augmented['image']
            mask_indexed = augmented['mask']
        else:
            mask_indexed = torch.from_numpy(mask_indexed).long()
        
        return {
            'image': image,
            'mask': mask_indexed,
            'filename': img_name
        }

print("SegmentationDataset class defined")

SegmentationDataset class defined


## 6. Calculate Class Weights for Imbalanced Dataset

In [6]:
def calculate_class_weights(mask_dir, num_classes=10):
    """Calculate class weights for imbalanced dataset"""
    class_counts = np.zeros(num_classes)
    
    mask_files = [f for f in os.listdir(mask_dir) if f.endswith('.png')]
    
    for mask_file in mask_files:
        mask = np.array(Image.open(os.path.join(mask_dir, mask_file)))
        
        # Count pixels for each class
        for class_id, class_idx in CLASS_MAPPING.items():
            class_counts[class_idx] += np.sum(mask == class_id)
    
    # Calculate weights (inverse frequency)
    total_pixels = class_counts.sum()
    class_weights = total_pixels / (class_counts * num_classes + 1e-6)
    
    # Normalize weights
    class_weights = class_weights / class_weights.sum() * num_classes
    
    return class_weights

# Calculate class weights
print("Calculating class weights...")
class_weights = calculate_class_weights(TRAIN_MASK_PATH)

print("\nClass Weights (for imbalanced dataset):")
for class_idx, weight in enumerate(class_weights):
    print(f"  {CLASS_NAMES[class_idx]}: {weight:.4f}")

# Save class weights
np.save('d:\\Semantic_Segmentation\\class_weights.npy', class_weights)

Calculating class weights...

Class Weights (for imbalanced dataset):
  Trees: 0.1798
  Lush Bushes: 0.1071
  Dry Grass: 0.0337
  Dry Bushes: 0.5791
  Ground Clutter: 0.1447
  Flowers: 0.2264
  Logs: 8.1554
  Rocks: 0.5308
  Landscape: 0.0260
  Sky: 0.0169

Class Weights (for imbalanced dataset):
  Trees: 0.1798
  Lush Bushes: 0.1071
  Dry Grass: 0.0337
  Dry Bushes: 0.5791
  Ground Clutter: 0.1447
  Flowers: 0.2264
  Logs: 8.1554
  Rocks: 0.5308
  Landscape: 0.0260
  Sky: 0.0169


## 7. Create Data Loaders

In [7]:
# Create datasets
print("Creating datasets...")
train_dataset = SegmentationDataset(
    img_dir=TRAIN_IMG_PATH,
    mask_dir=TRAIN_MASK_PATH,
    transform=train_transform
)

val_dataset = SegmentationDataset(
    img_dir=VAL_IMG_PATH,
    mask_dir=VAL_MASK_PATH,
    transform=val_transform
)

print(f"Training dataset: {len(train_dataset)} samples")
print(f"Validation dataset: {len(val_dataset)} samples")

# Create data loaders
train_loader = DataLoader(
    train_dataset,
    batch_size=CONFIG['batch_size'],
    shuffle=True,
    num_workers=CONFIG['num_workers'],
    pin_memory=True
)

val_loader = DataLoader(
    val_dataset,
    batch_size=CONFIG['batch_size'],
    shuffle=False,
    num_workers=CONFIG['num_workers'],
    pin_memory=True
)

print(f"\nDataLoader created:")
print(f"Training batches: {len(train_loader)}")
print(f"Validation batches: {len(val_loader)}")
print(f"Batch size: {CONFIG['batch_size']}")

Creating datasets...
Training dataset: 2857 samples
Validation dataset: 317 samples

DataLoader created:
Training batches: 179
Validation batches: 20
Batch size: 16


# Visualize augmented samples
fig, axes = plt.subplots(3, 4, figsize=(16, 12))

sample_idx = 0
sample = train_dataset[sample_idx]

# Show original and 3 augmented versions
for aug_idx in range(3):
    sample = train_dataset[sample_idx]
    
    image = sample['image'].numpy().transpose(1, 2, 0)
    # Denormalize
    image = image * np.array(NORM_STD).reshape(1, 1, 3) + np.array(NORM_MEAN).reshape(1, 1, 3)
    image = np.clip(image, 0, 1)
    
    mask = sample['mask'].numpy()
    
    axes[aug_idx, 0].imshow(image)
    axes[aug_idx, 0].set_title(f'Augmented Image {aug_idx+1}')
    axes[aug_idx, 0].axis('off')
    
    # Visualize mask with colors
    mask_colored = np.zeros((mask.shape[0], mask.shape[1], 3))
    colors = plt.cm.tab10(np.linspace(0, 1, CONFIG['num_classes']))
    for c in range(CONFIG['num_classes']):
        mask_colored[mask == c] = colors[c, :3]
    
    axes[aug_idx, 1].imshow(mask_colored)
    axes[aug_idx, 1].set_title(f'Augmented Mask {aug_idx+1}')
    axes[aug_idx, 1].axis('off')
    
    axes[aug_idx, 2].imshow(image * 0.6 + mask_colored * 0.4)
    axes[aug_idx, 2].set_title(f'Overlay {aug_idx+1}')
    axes[aug_idx, 2].axis('off')
    
    # Show class distribution in mask
    unique, counts = np.unique(mask, return_counts=True)
    class_dist = []
    for c in range(CONFIG['num_classes']):
        if c in unique:
            idx = np.where(unique == c)[0][0]
            class_dist.append(counts[idx])
        else:
            class_dist.append(0)
    
    axes[aug_idx, 3].bar(range(CONFIG['num_classes']), class_dist)
    axes[aug_idx, 3].set_title(f'Class Distribution {aug_idx+1}')
    axes[aug_idx, 3].set_xlabel('Class')
    axes[aug_idx, 3].set_ylabel('Pixel Count')
    axes[aug_idx, 3].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.savefig('augmented_samples.png', dpi=300, bbox_inches='tight')
plt.show()

print("Augmented samples visualization saved as 'augmented_samples.png'")

## 9. Save Configuration for Next Notebooks

In [8]:
import json

# Prepare configuration for model training
config_dict = {
    'image_size': CONFIG['img_size'][0],
    'num_classes': CONFIG['num_classes'],
    'normalize_mean': NORM_MEAN.tolist(),
    'normalize_std': NORM_STD.tolist(),
    'class_mapping': CLASS_MAPPING,
    'class_names': CLASS_NAMES,
    'batch_size': CONFIG['batch_size']
}

# Save configuration
config_path = 'd:\\Semantic_Segmentation\\preprocessing_config.json'
with open(config_path, 'w') as f:
    json.dump(config_dict, f, indent=2)

print(f"✓ Configuration saved to {config_path}")
print(f"\nFeature Engineering Complete!")
print(f"Generated files:")
print(f"  - class_weights.npy")
print(f"  - preprocessing_config.json")
print(f"  - augmented_samples.png")

✓ Configuration saved to d:\Semantic_Segmentation\preprocessing_config.json

Feature Engineering Complete!
Generated files:
  - class_weights.npy
  - preprocessing_config.json
  - augmented_samples.png
